In [45]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from pdfminer.high_level import extract_text
import os
import fitz
import time
import torch
import numpy as np
import re
import time
from transformers import AutoTokenizer, AutoModel
from pydantic import BaseModel, Field, PrivateAttr, ConfigDict
from typing import List, Tuple


In [46]:

model_name = "Qwen/Qwen2.5-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.37s/it]


In [28]:
class Process_pdf:
    def __init__(self, pdf_input_path, output_dir="./", chunk_len=500):
        self.pdf_input_path=pdf_input_path
        self.output_dir = output_dir 
        self.chunk_len = chunk_len 
        
        self.file_name = self.pdf_input_path.split('/')[-1].split('.')[0]
        if not os.path.exists(os.path.join(self.output_dir, self.file_name)):
            os.makedirs(os.path.join(self.output_dir, self.file_name))
    
    def extract_text(self):
        texts = []
        pdf_document = fitz.open(self.pdf_input_path)
        for page_num in range(len(pdf_document)):
            page = pdf_document.load_page(page_num)
            text = page.get_text("text")
            texts.append(text)
        pdf_document.close()
        text_pdf ="".join(texts)
        return text_pdf

def run_test(path_pdf):
    processer=Process_pdf(path_pdf)
    start = time.time()
    re=processer.extract_text()
    return re
    
def processing_document(filepath):
    text = ''
    for path_pdf in filepath: 
        text = text + run_test(path_pdf)
    return text

In [42]:
def clean_special_text(text):
    pattern = r'[^a-zA-Z0-9\s.,]'
    cleaned_text = re.sub(pattern, '', text)
    cleaned_text = cleaned_text.replace('\n', '')
    return cleaned_text
filepath = '/home/nhatthuong/Documents/Thesis/Acne-detection-and-treatment-recommendations/backend/fastapi_all/ai/rag/documents/'
documents = [os.path.join(filepath, filename) for filename in os.listdir(filepath)]
result = clean_special_text(processing_document(documents))
result


'NGUYEN NHAT THUONG Ho Chi Minh city  thuongnn525gmail.com  0342888525  httpsgithub.comngnhthuong Education  University of Information Technology VNUHCM  UIT      Oct 2020  March 2025 Expected Specialized Information System Advanced program  GPA 3.54 Scholarships Encouraging Study 2020, 2021, 2023, OEP Partial 2022 AI Viet Nam Course AIO2024         Feb 2024  May 2025 Expected  Specialized Computer vision, Natural language processing  Experience  Data science Metamind Course                                                          Feb 2024  Augst 2024 Teaching Assistant  Teaching the basics of machine learning and deep learning to a course with 81 members.  Project advisor for learning modules.  Projects       Glowypa  Skincare with AI  Deep learning, RAG, LLMs, YOLOv8, SAHI, Object Detection, Reactjs, Canvas, FastAPI, MongoDB Demo glowypa.com Github httpsgithub.comngnhthuongAcnedetectionandtreatmentrecommendations.git  Project Objective Created an AI system that can identify and categ

In [43]:
class TextProcessor(BaseModel):
    text: str

    def split_sentences(self) -> List[str]:
        sentences = re.split(r'(?<=[.?!])\s+', self.text.strip())
        return sentences

    def combine_sentences(self, sentences: List[str]) -> List[str]:
        combined_sentences = []
        for i in range(len(sentences)):
            combined_sentence = sentences[i]
            if i > 0:
                combined_sentence = sentences[i - 1] + ' ' + combined_sentence
            if i < len(sentences) - 1:
                combined_sentence += ' ' + sentences[i + 1]
            combined_sentences.append(combined_sentence)
        return combined_sentences

class EmbeddingModel(BaseModel):
    model_name: str = Field(default="sentence-transformers/all-MiniLM-L6-v2")
    _tokenizer: AutoTokenizer = PrivateAttr()
    _model: AutoModel = PrivateAttr()
    model_config = ConfigDict(arbitrary_types_allowed=True)
    
    def __init__(self, **data):
        super().__init__(**data)
        self._tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self._model = AutoModel.from_pretrained(self.model_name)

    def get_embeddings(self, texts: List[str]) -> np.ndarray:
        encoded_input = self._tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            model_output = self._model(**encoded_input)
        embeddings = self._mean_pooling(model_output, encoded_input['attention_mask'])
        return embeddings.numpy()
    
    @staticmethod
    def _mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[0]  # First element of output is the embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        summed_tokens = torch.sum(token_embeddings * input_mask_expanded, dim=1)
        summed_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
        return summed_tokens / summed_mask

class CosineDistanceCalculator(BaseModel):
    embeddings: np.ndarray
    model_config = ConfigDict(arbitrary_types_allowed=True)

    def calculate_distances(self) -> np.ndarray:
        # Normalize embeddings
        norms = np.linalg.norm(self.embeddings, axis=1, keepdims=True)
        normalized_embeddings = self.embeddings / norms

        # Compute cosine similarities between consecutive embeddings
        cosine_sims = np.sum(normalized_embeddings[:-1] * normalized_embeddings[1:], axis=1)
        distances = 1 - cosine_sims
        return distances

class TextChunker(BaseModel):
    text_processor: TextProcessor
    embedding_model: EmbeddingModel
    breakpoint_percentile_threshold: float = Field(default=80.0)
    chunks: List[str] = Field(default_factory=list)
    chunk_embeddings: np.ndarray = Field(default=None)
    _distances: np.ndarray = PrivateAttr(default=None)
    model_config = ConfigDict(arbitrary_types_allowed=True)

    def chunk_text(self):
        single_sentences = self.text_processor.split_sentences()

        combined_sentences = self.text_processor.combine_sentences(single_sentences)

        embeddings = self.embedding_model.get_embeddings(combined_sentences)

        distance_calculator = CosineDistanceCalculator(embeddings=embeddings)
        distances = distance_calculator.calculate_distances()
        self._distances = distances  # Store distances if needed later

        # Determine the threshold
        breakpoint_distance_threshold = np.percentile(distances, self.breakpoint_percentile_threshold)

        # Find indices where the distance exceeds the threshold
        indices_above_thresh = [i for i, distance in enumerate(distances) if distance > breakpoint_distance_threshold]

        start_index = 0
        for index in indices_above_thresh:
            chunk = ' '.join(single_sentences[start_index:index + 1])
            self.chunks.append(chunk)
            start_index = index + 1
            
        # Add any remaining sentences as the last chunk
        if start_index < len(single_sentences):
            chunk = ' '.join(single_sentences[start_index:])
            self.chunks.append(chunk)
            
        # Compute embeddings for the chunks
        self.chunk_embeddings = self.embedding_model.get_embeddings(self.chunks)

    def get_chunks(self) -> List[str]:
        return self.chunks

class SemanticSearchEngine(BaseModel):
    embedding_model: EmbeddingModel
    chunks: List[str]
    chunk_embeddings: np.ndarray
    model_config = ConfigDict(arbitrary_types_allowed=True)

    def search(self, query: str, top_k: int = 3) -> List[Tuple[str, float]]:
        query_embedding = self.embedding_model.get_embeddings([query])[0]
        # Compute cosine similarities
        cosine_similarities = self._compute_cosine_similarities(query_embedding, self.chunk_embeddings)
        # Get top_k highest similarity scores
        top_k_indices = np.argsort(cosine_similarities)[::-1][:top_k]
        top_chunks = [(self.chunks[i], cosine_similarities[i]) for i in top_k_indices]
        return top_chunks

    @staticmethod
    def _compute_cosine_similarities(query_embedding: np.ndarray, chunk_embeddings: np.ndarray) -> np.ndarray:
        # Normalize embeddings
        query_norm = np.linalg.norm(query_embedding)
        chunk_norms = np.linalg.norm(chunk_embeddings, axis=1)

        # Compute dot product
        dot_products = np.dot(chunk_embeddings, query_embedding)

        # Compute cosine similarities
        cosine_similarities = dot_products / (chunk_norms * query_norm + 1e-10)
        return cosine_similarities

start_time = time.time()
text = result
text_processor = TextProcessor(text=text)
embedding_model = EmbeddingModel()
text_chunker = TextChunker(
    text_processor=text_processor,
    embedding_model=embedding_model,
    breakpoint_percentile_threshold=80.0 #threshold 
)
text_chunker.chunk_text()
chunks = text_chunker.get_chunks()
end_time = time.time()
elapsed_time = end_time - start_time
print("-------------------------------------RESULT SEMANTIC CHUNKING---------------------------------------")
for i, chunk in enumerate(chunks, 1):
    print(f"\nChunk {i}:")
    print(chunk)
print(f"\nTotal number of chunks: {len(chunks)}")

search_engine = SemanticSearchEngine(
    embedding_model=embedding_model,
    chunks=text_chunker.chunks,
    chunk_embeddings=text_chunker.chunk_embeddings
)
query = "How to treatment whitehead?"
top_results = search_engine.search(query, top_k=1) #get 1 top_k
print("-------------------------------------RESULT SEARCH--------------------------------------------------")
print(f"\nTop results for query: '{query}'")
for idx, (chunk, score) in enumerate(top_results, 1):
    print(f"\nResult {idx}:")
    print(f"Similarity Score: {score:.4f}")
    print(chunk)
print("-----------------------------------------TIME USED--------------------------------------------------")
print("Time used: {:.2f} second".format(elapsed_time))


/home/nhatthuong/.miniconda3/envs/skin_env/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in EmbeddingModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


-------------------------------------RESULT SEMANTIC CHUNKING---------------------------------------

Chunk 1:
NGUYEN NHAT THUONG Ho Chi Minh city  thuongnn525gmail.com  0342888525  httpsgithub.comngnhthuong Education  University of Information Technology VNUHCM  UIT      Oct 2020  March 2025 Expected Specialized Information System Advanced program  GPA 3.54 Scholarships Encouraging Study 2020, 2021, 2023, OEP Partial 2022 AI Viet Nam Course AIO2024         Feb 2024  May 2025 Expected  Specialized Computer vision, Natural language processing  Experience  Data science Metamind Course                                                          Feb 2024  Augst 2024 Teaching Assistant  Teaching the basics of machine learning and deep learning to a course with 81 members.

Chunk 2:
Project advisor for learning modules. Projects       Glowypa  Skincare with AI  Deep learning, RAG, LLMs, YOLOv8, SAHI, Object Detection, Reactjs, Canvas, FastAPI, MongoDB Demo glowypa.com Github httpsgithub.comngnh

In [58]:
prompt = ""
query = "How to Treatment whitehead?"
top_results = search_engine.search(query, top_k=5) #get 1 top_k
messages = [
    {"role": "system", "content": f"{top_results}" + "Please response by Vietnamese"},
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [59]:
print(response)

Dưới đây là thông tin về các phương pháp điều trị được liệt kê trong tài liệu, nhưng đã được dịch sang tiếng Việt:

1. **Các loại mụn và cách nhận biết:**
   - Nốt đen (Blackhead): Lỗ chân lông bị tắc do sebum và tế bào da chết, gây đen do tiếp xúc với không khí.
   - Nốt trắng (Whitehead): Lỗ chân lông bị tắc nhưng không tiếp xúc với không khí, hình thành thành những nốt trắng nhỏ.
   
   **Chữa trị:**
   - Ascorbic Acid (Salicylic Acid): Chăm sóc sâu vào lỗ chân lông để làm sạch và giảm viêm.
   - Benzoyl Peroxide: Loại bỏ vi khuẩn gây mụn.
   - Retinoids: Giúp ngăn chặn lỗ chân lông bị tắc và thúc đẩy quá trình tái tạo tế bào da.
   - Làm sạch bằng tay (Physical Exfoliants): Loại bỏ tế bào da chết trên bề mặt.

   **Sản phẩm phù hợp:** Kem làm sạch chứa Ascorbic Acid, Mask đất sét.

2. **Mụn cong bunda (Conglobata Acne):**
   - Nhận biết: Mụn nặng với các nodule lớn và sâu, thường đau và có thể để lại vết thương hở.
   - Điều trị:
     - Isotretinoin: Thuốc uống mạnh mẽ và hiệu quả 

In [60]:
import markdown

def chatgpt_response_to_html(response_text):
    """
    Convert ChatGPT's response into HTML formatted text.
    Args:
        response_text (str): The text response from ChatGPT.
    Returns:
        str: HTML formatted string.
    """
    html_output = markdown.markdown(response_text)
    return html_output

In [61]:
chatgpt_response_to_html(response)

'<p>Dưới đây là thông tin về các phương pháp điều trị được liệt kê trong tài liệu, nhưng đã được dịch sang tiếng Việt:</p>\n<ol>\n<li><strong>Các loại mụn và cách nhận biết:</strong></li>\n<li>Nốt đen (Blackhead): Lỗ chân lông bị tắc do sebum và tế bào da chết, gây đen do tiếp xúc với không khí.</li>\n<li>Nốt trắng (Whitehead): Lỗ chân lông bị tắc nhưng không tiếp xúc với không khí, hình thành thành những nốt trắng nhỏ.</li>\n</ol>\n<p><strong>Chữa trị:</strong>\n   - Ascorbic Acid (Salicylic Acid): Chăm sóc sâu vào lỗ chân lông để làm sạch và giảm viêm.\n   - Benzoyl Peroxide: Loại bỏ vi khuẩn gây mụn.\n   - Retinoids: Giúp ngăn chặn lỗ chân lông bị tắc và thúc đẩy quá trình tái tạo tế bào da.\n   - Làm sạch bằng tay (Physical Exfoliants): Loại bỏ tế bào da chết trên bề mặt.</p>\n<p><strong>Sản phẩm phù hợp:</strong> Kem làm sạch chứa Ascorbic Acid, Mask đất sét.</p>\n<ol>\n<li><strong>Mụn cong bunda (Conglobata Acne):</strong></li>\n<li>Nhận biết: Mụn nặng với các nodule lớn và sâu, 

In [11]:
from langchain_community.chat_models import ChatOllama
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage

# Khởi tạo chat model - sử dụng tên đầy đủ của model
chat = ChatOllama(
    model="llama3.2:latest",  # Thêm ":latest" vào tên model
    temperature=0.7
)

# Test chat đơn giản
messages = [
    SystemMessage(content="Bạn là một trợ lý AI hữu ích."),
    HumanMessage(content="Xin chào!")
]

try:
    response = chat.invoke(messages)
    print(response.content)
except Exception as e:
    print(f"Lỗi: {e}")


Lỗi: Ollama call failed with status code 500. Details: {"error":"llama runner process has terminated: exit status 127"}


In [7]:
import os

folder_path = "/home/nhatthuong/Documents/Thesis/Acne-detection-and-treatment-recommendations/backend/fastapi_all/ai/rag/doctor_advice/storage"
files = os.listdir(folder_path)
file_paths = []
for file in files:
    file_path = os.path.join(folder_path, file)
    file_paths.append(file_path)
print(file_paths)


['/home/nhatthuong/Documents/Thesis/Acne-detection-and-treatment-recommendations/backend/fastapi_all/ai/rag/doctor_advice/storage/Acne_Vulgaris.pdf', '/home/nhatthuong/Documents/Thesis/Acne-detection-and-treatment-recommendations/backend/fastapi_all/ai/rag/doctor_advice/storage/acne_treatment_2.pdf']


In [2]:
import ollama
response = ollama.chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

ConnectError: [Errno 111] Connection refused

In [15]:
ollama.chat(model='llama3.1', messages=[{'role': 'user', 'content': 'Why is the sky blue?'}])


ResponseError: model "llama3.1" not found, try pulling it first